In [63]:
!pip install selenium

In [23]:
#final

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import re
from datetime import datetime, timedelta


State_Name_List = []
Bus_Route_List = []
Route_Link_List = []
data = []
data1 = []
f=0



def generate_url(base_url, days_offset=1):
    # Get today's date or any offset (days from today)
    target_date = datetime.now() + timedelta(days=days_offset)
    
    # Format the date as "DD-MMM-YYYY"
    formatted_date = target_date.strftime("%d-%b-%Y")
    
    # Replace the placeholder with the formatted date in the base URL
    dynamic_url = base_url.replace("23-Sep-2024", formatted_date)
    
    return dynamic_url

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)  # Adjust time if needed
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver = webdriver.Chrome()
driver.get("https://www.redbus.in")
driver.maximize_window()
time.sleep(5)
driver.execute_script("window.scrollBy(0, 700);")

wait = WebDriverWait(driver, 10)

State_Name = driver.find_elements(By.XPATH, "//div[@class='rtcBack']")
for i in State_Name:
    text = i.text
    State_Name_List.append(text.split('\n')[0])


df = pd.DataFrame(State_Name_List, columns=["Bus_State"])

# Clean up state names
df['Bus_State'] = df['Bus_State'].str.lower()
df['Bus_State'] = df['Bus_State'].replace({
    'kerala rtc': 'ksrtc-kerala',
    'ctu rtc': 'chandigarh-transport-undertaking-ctu',
    'kaac transport': 'kaac-transport',
    'sbstc': 'south-bengal-state-transport-corporation-sbstc',
    'wbtc': 'wbtc-ctc',
     'tgsrtc':'tsrtc',
    'nbstc': 'north-bengal-state-transport-corporation',
    'bsrtc': 'bihar-state-road-transport-corporation-bsrtc',
    'wbstc': 'west-bengal-transport-corporation'
})


def scrape_page(state):
    try:
        # Wait until the route details are present on the page
        routes_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_details")))

        # Loop through each route to extract details
        for route in routes_container:
            try:
                # Clean text by removing prices and extra spaces
                cleaned_text = re.sub(r'From INR \d+', '', route.text).strip()
                cleaned_text = "\n".join([line.strip() for line in cleaned_text.splitlines() if line.strip()])

                # Get the route link
                try:
                    rt = driver.find_element(By.XPATH, f"//a[@title='{cleaned_text}']")
                    link = rt.get_attribute("href")
                    Route_Link_List.append(link)
                    df1 = pd.DataFrame(Route_Link_List, columns=["RouteLink"])
              
                    


                except:
                    link = "No link available"
                # Append the data for each route
                data.append({
                    "StateName": state,
                    "BusRoute": cleaned_text,
                    "RouteLink": link
                })

            except Exception as e:
                print(f"Error processing route: {e}")
                continue

    except Exception as e:
        print(f"Error processing state {state}: {e}")
a=0 

            
for state in df['Bus_State']:
        driver.get(f"https://www.redbus.in/online-booking/{state}")
        time.sleep(10)
        #scrape_page(state)
        a=a+1
        print(f"{a} {state}")
        # Scrape data from the first 5 pages
        for page_number in range(1, 6):
            scrape_page(state)
            if page_number < 5:  # Don't try to click next on the last page
                try:
                    # Locate the pagination container
                    pagination_container = wait.until(EC.presence_of_element_located(
                        (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
                    ))
        
                    next_page_button = pagination_container.find_element(
                        By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
                    )
        
                    # Ensure the next page button is in view
                    actions = ActionChains(driver)
                    actions.move_to_element(next_page_button).perform()
                    time.sleep(1)  # Wait for a bit after scrolling
        
                    # Log the action
                    #print(f"Clicking on page {page_number + 1}")
        
                    # Click the next page button
                    next_page_button.click()
        
                    # Wait for the page number to update to the next page
                    wait.until(EC.text_to_be_present_in_element(
                        (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
                    # Log the successful page navigation
                   # print(f"Successfully navigated to page {page_number + 1}")
        
                    # Wait for a short duration to ensure the next page loads completely
                    time.sleep(3)
                except Exception as e:
                   
                    break


        



for link in Route_Link_List:
    driver.get(link)

           
    f=f+1
           
                    
    time.sleep(15)
    From_To= driver.find_element(By.CLASS_NAME, "D136_h1")
    From_To_txt=From_To.text
    print(f"{f},{From_To_txt}")
    ViewBus_Btn = driver.find_elements(By.XPATH, "(//div[@class='button'])[1]")
            
                          
    if ViewBus_Btn:
        ViewBus_Btn[0].click()  # Click the first button in the list
        time.sleep(5)
        
        scroll()  # Scroll the page after clicking
        BusNames = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        BusTypes=driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        Bus_Departure_Time=driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        Bus_Reaching_Time=driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        Bus_Rating=driver.find_elements(By.XPATH, "//div[@class='lh-18 rating rat-green ']")
        Ticket_Price=driver.find_elements(By.XPATH, "//div[@class='fare d-block']")
        Seat_Availability=driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-16']")
        Bus_Duration=driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")

                        
    else:
        scroll()
        time.sleep(5)
        
        BusNames = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        BusTypes=driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        Bus_Departure_Time=driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        Bus_Reaching_Time=driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        Bus_Rating=driver.find_elements(By.XPATH, "//div[@class='lh-18 rating rat-green ']")
        Ticket_Price=driver.find_elements(By.XPATH, "//div[@class='fare d-block']")
        Seat_Availability=driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-16']")
        Bus_Duration=driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")

        
        
            
        
        
            
        
    for  name, btype, bdep, breach,bduration, brating, tprice, savailability in zip(BusNames,BusTypes,Bus_Departure_Time,Bus_Reaching_Time,Bus_Duration,Bus_Rating,Ticket_Price,Seat_Availability):
                    
        BusName_txt = name.text
        BusType_txt = btype.text
        Bus_Departure_Time=bdep.text
        Bus_Reaching_Time=breach.text
        Bus_Duration=bduration.text

        Bus_Rating=brating.text
        Ticket_Price=tprice.text
        Seat_Availability=savailability.text
                            
        data1.append({
                                    
            "Route_Name": From_To_txt,
            "RouteLink":link,
            "BusName": BusName_txt,
            "BusType": BusType_txt,
            "BusDepartureTime":Bus_Departure_Time,
            "BusReachingTime":Bus_Reaching_Time,
            "Bus_Duration":Bus_Duration,

            "BusRating":Bus_Rating,
            "TicketPrice":Ticket_Price,
            "SeatAvailability":Seat_Availability
                                    
                                
                                })
        result_df1 = pd.DataFrame(data1)
        result_df1['AvailableSeats'] = result_df1['SeatAvailability'].str.extract('(\d+)')        

        # Split the 'Route_Name' column by the string " to " and create 'From_Place' and 'To_Place' columns
        result_df1[['From_Place', 'To_Place']] = result_df1['Route_Name'].str.split(' to ', expand=True)

# Drop the "Bus" part from 'To_Place' if it exists (i.e., remove the word "Bus" at the end)
        result_df1['To_Place'] = result_df1['To_Place'].str.replace(' Bus', '', regex=False)

# Display the updated DataFrame
        #print(result_df1[['Route_Name', 'From_Place', 'To_Place']])
        result_df1.to_csv('Bus_Details_data.csv', index=False)
         
driver.quit()

<>:243: SyntaxWarning: invalid escape sequence '\d'
<>:243: SyntaxWarning: invalid escape sequence '\d'
C:\Users\DELL\AppData\Local\Temp\ipykernel_17208\2104580875.py:243: SyntaxWarning: invalid escape sequence '\d'
  result_df1['AvailableSeats'] = result_df1['SeatAvailability'].str.extract('(\d+)')


1 apsrtc
2 ksrtc-kerala
3 tsrtc
4 ktcl
5 rsrtc
6 south-bengal-state-transport-corporation-sbstc
7 hrtc
8 astc
9 upsrtc
10 wbtc-ctc
11 chandigarh-transport-undertaking-ctu
12 pepsu
13 north-bengal-state-transport-corporation
14 bihar-state-road-transport-corporation-bsrtc
15 kaac-transport
16 west-bengal-transport-corporation
17 jksrtc
1,Hyderabad to Vijayawada Bus
2,Vijayawada to Hyderabad Bus


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.60)
Stacktrace:
	GetHandleVerifier [0x00007FF63E00B645+29573]
	(No symbol) [0x00007FF63DF80470]
	(No symbol) [0x00007FF63DE3B6EA]
	(No symbol) [0x00007FF63DE0FCD5]
	(No symbol) [0x00007FF63DEBEF67]
	(No symbol) [0x00007FF63DED7FC1]
	(No symbol) [0x00007FF63DEB70A3]
	(No symbol) [0x00007FF63DE812DF]
	(No symbol) [0x00007FF63DE82441]
	GetHandleVerifier [0x00007FF63E33C58D+3375821]
	GetHandleVerifier [0x00007FF63E387987+3684039]
	GetHandleVerifier [0x00007FF63E37CDAB+3640043]
	GetHandleVerifier [0x00007FF63E0CB7C6+816390]
	(No symbol) [0x00007FF63DF8B77F]
	(No symbol) [0x00007FF63DF875A4]
	(No symbol) [0x00007FF63DF87740]
	(No symbol) [0x00007FF63DF7659F]
	BaseThreadInitThunk [0x00007FFFEB667374+20]
	RtlUserThreadStart [0x00007FFFECD1CC91+33]


In [24]:
df_data = pd.DataFrame(data)

final_df = pd.merge(df_data, result_df1, on='RouteLink', how='inner')

print(final_df)

   StateName                 BusRoute  \
0     apsrtc  Hyderabad to Vijayawada   
1     apsrtc  Hyderabad to Vijayawada   
2     apsrtc  Hyderabad to Vijayawada   
3     apsrtc  Hyderabad to Vijayawada   
4     apsrtc  Hyderabad to Vijayawada   
..       ...                      ...   
93     tsrtc  Hyderabad to Vijayawada   
94     tsrtc  Hyderabad to Vijayawada   
95     tsrtc  Hyderabad to Vijayawada   
96     tsrtc  Hyderabad to Vijayawada   
97     tsrtc  Hyderabad to Vijayawada   

                                            RouteLink  \
0   https://www.redbus.in/bus-tickets/hyderabad-to...   
1   https://www.redbus.in/bus-tickets/hyderabad-to...   
2   https://www.redbus.in/bus-tickets/hyderabad-to...   
3   https://www.redbus.in/bus-tickets/hyderabad-to...   
4   https://www.redbus.in/bus-tickets/hyderabad-to...   
..                                                ...   
93  https://www.redbus.in/bus-tickets/hyderabad-to...   
94  https://www.redbus.in/bus-tickets/hyderabad-to.

In [13]:
final_df.to_csv(r'C:\Users\DELL\Desktop\RedBus\Final_Bus_Details.csv', index=False)
# a = pd.read_csv('Bus_Details_data.csv')
# a

In [29]:
import mysql.connector
import pandas as pd

# MySQL Database connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    # Replace 'test' with the actual database name
    database='test'  
)

print(mydb)

# Ensure the connection was successful
if mydb.is_connected():
    print("Successfully connected to the database")

# Create a cursor object
mycursor = mydb.cursor(buffered=True)

# Use the correct database
mycursor.execute("USE redbusdata")

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS final_bus_details (
    id INT AUTO_INCREMENT PRIMARY KEY,
    StateName VARCHAR(255),
    From_Place VARCHAR(255),
    To_Place VARCHAR(255),
    Route_Name VARCHAR(255),
    RouteLink VARCHAR(255),
    BusName VARCHAR(255),
    BusType VARCHAR(50),
    BusDepartureTime VARCHAR(50),
    BusReachingTime VARCHAR(50),
    Bus_Duration VARCHAR(50),
    BusRating VARCHAR(10),
    TicketPrice VARCHAR(50),
    AvailableSeats VARCHAR(50)
)
"""
mycursor.execute(create_table_query)

# Insert query
insert_query = """
INSERT INTO final_bus_details (
    StateName, From_Place, To_Place, Route_Name, RouteLink, BusName, BusType, BusDepartureTime, 
    BusReachingTime, Bus_Duration, BusRating, TicketPrice, AvailableSeats
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Assuming final_df is a Pandas DataFrame, iterate through it using .iterrows()
for index, record in final_df.iterrows():
    values = (
        record['StateName'],
        record['From_Place'],
        record['To_Place'],
        record['Route_Name'],
        record['RouteLink'],
        record['BusName'],
        record['BusType'],
        record['BusDepartureTime'],
        record['BusReachingTime'],
        record['Bus_Duration'],
        record['BusRating'],
        record['TicketPrice'],
        record['AvailableSeats']
    )
    
    mycursor.execute(insert_query, values)

# Commit the transaction
mydb.commit()

# Confirm number of rows inserted
print(f"{mycursor.rowcount} records inserted.")

# Close the cursor and connection
mycursor.close()
mydb.close()


Successfully connected to the database
1 records inserted.
